In [71]:
! pip install transformers
! pip install selenium

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [72]:
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/grap

In [73]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
# 패키지 import
import os
from time import sleep



import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
d = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

'''
s = Service('/content/chromedriver.exe')

d = webdriver.Chrome(service = s)

'''

name = ['에어팟 1세대', '에어팟 2세대']


# 쇼핑몰 검색
link = "https://search.shopping.naver.com/catalog/16233352105?NaPm=ct%3Dl1vtwy7k%7Cci%3Da2cc9438d6415f5c49377e2e7760e09349687ad4%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3Db9076c74953a2f0f7ddee6918d7b29798432fa16"
# xpath
shoppingmall_review = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[2]/div/div[2]/ul/li[3]/a/strong"
all_review = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[1]/a"

header = {'User-Agent': ''}

d.implicitly_wait(3)
d.get(link)
req = requests.get(link)
html = req.text
soup = BeautifulSoup(html, "html.parser")


# 함수 선언
def add_dataframe(stars, reviews, cnt):  # 데이터 프레임에 저장
    # 데이터 프레임생성
    df1 = pd.DataFrame(columns=['star', 'review'])
    n = 1
    if (cnt > 0):
        for i in range(0, cnt - 1):
            df1.loc[n] = [stars[i],reviews[i]]  # 해당 행에 저장
            i += 1
            n += 1
    else:
        df1.loc[n] = ['null', 'null']
        n += 1
    return df1


def save():
    if not os.path.exists('output1.csv'):
        df1.to_csv('output1.csv', encoding='utf-8', mode='w')
    else:
        df1.to_csv('output1.csv', encoding='utf-8', mode='w', header=False)


# 쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review)

sleep(2)

# 전체 리뷰 가져오기
d.find_element(By.XPATH, all_review).click()  # 스크롤 건드리면 안됨
name_ = name[0]  # 에어팟 1세대
reviews = []
stars = []
cnt = 1  # 리뷰index
page = 1
while True:
    j = 1
    print("페이지", page, "\n")
    sleep(2)
    while True:  # 한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[' + str(j) +']/div[1]/span[1]').text
            stars.append(star)
            review = d.find_element(By.XPATH,
               '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[' + str(j) +']/div[2]/div[1]/p').text
            reviews.append(review)
            if j % 2 == 0:  # 화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element(By.XPATH,
                    '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li[' + str(j) +']/div[2]/div[1]/p')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
            j += 1
            #print(cnt, star, review, "\n")
            cnt += 1
        except: break

    sleep(2)

    if page < 11:  # page10까지 적용
        try:  # 리뷰의 마지막 페이지에서 error발생
            next_page = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[' + str(page) + ']').click()
            page += 1
        except: break  # 리뷰의 마지막 페이지에서 process 종료

    else:
        try:  # page11부터
            if page % 10 == 0:
                next_page = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else:
                next_page = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[' + str(page % 10 + 2) + ']').click()
            page += 1
        except: break

    if cnt > 40: break

  

df1 = add_dataframe(stars, reviews, cnt)
save()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


페이지 1 

페이지 2 



In [ ]:
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import requests
import time
import csv

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer
from keras.preprocessing.sequence import pad_sequences


## **전처리**

In [ ]:
# 데이터 로드
# 열 이름 추가해서 데이터로드
train = pd.read_csv("/content/output1.csv", names = ["index", "rating", "review","label"])
# test_data = pd.read_csv("")
print(train.shape)
#print(test.shape)


In [ ]:
train.head(10)


In [ ]:
# 리뷰 본문만 출력
sentence = train['Review']
sentence[:10]


In [ ]:
# [CLS]는 문장의 시작에 사용
# [SEP]는 문장의 끝이나 두 문장 분리에 사용

sentence = ["[CLS] " + str(i) + " [SEP]" for i in sentence ]
sentence[:10]

In [ ]:
#라벨 추출
# 우리는 데이터에 라벨을 추가해야함
labels = train['label'].values

In [ ]:
# Bert-base의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenized = [tokenizer.tokenize(i) for i in sentence]

print(tokenized[0])
 


In [ ]:
# 토큰을 숫자 인덱스로 변환
input = [tokenizer.convert_tokens_to_ids(i) for i in tokenized]

# 문장을 max_len 길이에 맞게 자르고 모자란 부분은 0으로 채움
input = pad_sequences(input, maxlen = 500, dtype = "long", truncating = "post", padding = "post")
input[0]

In [ ]:
# Attention mask 생성
# 패딩에 해당하는 부분은 0으로 패딩이 아닌 부분은 1로 간주하는 마스크 생성

# 어텐션 마스크 초기화
attention_masks = []

for se in input:
    seq_mask = [float(i>0) for i in se]
    attention_masks.append(seq_mask)

print(attention_masks[0])

In [ ]:
# 데이터를 텐서로 변환
# train 비율과 validation의 비율이 9대 1
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)





In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = 100)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=100)

## **모델 생성**

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
# 디바이스 설정
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")


In [ ]:
# BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

In [ ]:
# optimizer 설정 => optimizer는 AdamW

optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5)

epochs = 10

# 스케줄러 생성(학습률을 조금씩 감소)


In [ ]:
# 학습
for i in range(epochs):
    model.train()
    cost = 0 
    
    # 데이터로더에서 배치만큼 가져옴
    
    
